# Wind Turbine Power Estimation Interactive Demo

This notebook allows you to estimate expected turbine power output using:

- Weibull-distributed wind speeds
- Cubic spline power curve interpolation

You can input wind parameters interactively and visualize:

1. Wind speed distribution
2. Expected power generation
3. Comparison of generated samples vs. power curve

## Instructions

1. Adjust the wind parameters using the sliders or text boxes:
    - Mean wind speed (m/s)
    - Standard deviation of wind speed (m/s)
    - Minimum and maximum wind speeds (m/s)
    - Interval duration (minutes)
    - Number of synthetic samples
2. The analysis will calculate:
    - Weibull shape (k) and scale (λ) parameters
    - Generated wind speed samples
    - Expected average power (kW) and total energy (kWh)
3. Plots will show the wind distribution and expected power output.


Imports and Helper functions:

In [3]:
import numpy as np
from scipy.stats import weibull_min
from scipy.optimize import fsolve
from scipy.special import gamma
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from ipywidgets import interactive, FloatSlider, IntSlider, FloatText, IntText, HBox, VBox
from IPython.display import display

# --- Weibull ratio equation ---
def weibull_ratio_equation(k, ratio):
    return (gamma(1 + 2/k) - (gamma(1 + 1/k))**2) / (gamma(1 + 1/k))**2 - ratio**2

# --- Cubic spline power curve ---
def turbine_power_curve_from_data():
    wind_speeds = np.array([3, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18])
    power_outputs = np.array([26, 56, 130, 243, 404, 622, 894, 1175, 1395, 1485, 1491, 1498, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500, 1500])
    power_curve_func = CubicSpline(wind_speeds, power_outputs, bc_type='natural')
    def wrapped_power_curve(speed):
        if speed < wind_speeds[0] or speed > wind_speeds[-1]:
            return 0
        return power_curve_func(speed)
    return np.vectorize(wrapped_power_curve)


Analysis Function

In [4]:
def run_analysis(mean_wind_speed, std_dev, min_wind_speed, max_wind_speed, interval_minutes, num_samples):
    print("--- Starting Weibull Analysis ---")
    print(f"Inputs: Mean={mean_wind_speed:.2f} m/s, StdDev={std_dev:.2f} m/s, Min={min_wind_speed:.2f} m/s, Max={max_wind_speed:.2f} m/s")

    if std_dev == 0:
        print("\nStandard deviation is zero. Cannot calculate Weibull parameters.")
        return

    ratio = std_dev / mean_wind_speed
    k = fsolve(weibull_ratio_equation, 2.0, args=(ratio))[0]

    try:
        lambd = mean_wind_speed / gamma(1 + 1/k)
    except ZeroDivisionError:
        print("\nCould not calculate lambda. Check input values.")
        return

    print(f"\nCalculated Weibull Parameters: k={k:.4f}, lambda={lambd:.4f}")

    np.random.seed(42)
    weibull_dist = weibull_min(c=k, scale=lambd)
    wind_speed_samples = weibull_dist.rvs(size=num_samples)
    wind_speed_samples = wind_speed_samples[(wind_speed_samples >= min_wind_speed) & (wind_speed_samples <= max_wind_speed)]

    print(f"Generated {len(wind_speed_samples)} wind speed samples.")

    power_curve_func = turbine_power_curve_from_data()
    power_generated = power_curve_func(wind_speed_samples)

    average_power_kW = np.mean(power_generated)
    energy_kwh = average_power_kW * (interval_minutes / 60)

    print(f"\nExpected average power: {average_power_kW:.2f} kW")
    print(f"Total expected energy: {energy_kwh:.2f} kWh")

    plt.style.use('seaborn-v0_8-whitegrid')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Histogram + Weibull PDF
    ax1.hist(wind_speed_samples, bins=10, density=True, alpha=0.7, color='c', label='Generated Samples')
    x_vals = np.linspace(min_wind_speed, max_wind_speed, 100)
    ax1.plot(x_vals, weibull_dist.pdf(x_vals), 'r-', lw=2, label='Weibull PDF')
    ax1.set_title('Wind Speed Distribution')
    ax1.set_xlabel('Wind Speed (m/s)')
    ax1.set_ylabel('Probability Density')
    ax1.legend()

    # Power curve plot
    curve_speeds = np.linspace(0, 20, 300)
    ax2.plot(curve_speeds, power_curve_func(curve_speeds), 'b-', label='Power Curve')
    ax2.scatter(wind_speed_samples, power_generated, color='r', s=20, label='Sampled Points')
    ax2.set_title('Expected Power Output')
    ax2.set_xlabel('Wind Speed (m/s)')
    ax2.set_ylabel('Power (kW)')
    ax2.legend()
    ax2.set_ylim(bottom=0)

    plt.tight_layout()
    plt.show()


Interactive Widgets

In [5]:
# Sliders and text boxes for interactive input
mean_speed_slider = FloatSlider(min=0, max=30, step=0.1, value=11.96, description='Mean:')
std_dev_slider = FloatSlider(min=0, max=10, step=0.1, value=0.66, description='Std Dev:')
min_speed_slider = FloatSlider(min=0, max=30, step=0.1, value=9.3, description='Min:')
max_speed_slider = FloatSlider(min=0, max=30, step=0.1, value=15.13, description='Max:')
interval_slider = IntSlider(min=1, max=60, step=1, value=10, description='Interval (min):')
samples_slider = IntSlider(min=100, max=2000, step=100, value=600, description='Samples:')

output = interactive(
    run_analysis,
    mean_wind_speed=mean_speed_slider,
    std_dev=std_dev_slider,
    min_wind_speed=min_speed_slider,
    max_wind_speed=max_speed_slider,
    interval_minutes=interval_slider,
    num_samples=samples_slider
)

display(output)


interactive(children=(FloatSlider(value=11.96, description='Mean:', max=30.0), FloatSlider(value=0.66, descrip…